# Install JoinBoost and databases package.

To run this demo, you'll need to install the JoinBoost and databases packages. We will use DuckDB as the database in this demo.


In [3]:
%pip install joinboost==0.0.1523
%pip install duckdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 2.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Download and Load Data into a Database

This step is only Necessary for the Demo as pre-existing databases are unavailable. In general, JoinBoost enables direct model training over databases without data downloading or loading.


In [4]:
# Import required libraries
import duckdb
import urllib.request
from joinboost.executor import DuckdbExecutor
from joinboost.joingraph import JoinGraph
from joinboost.app import DecisionTree, GradientBoosting

# Define URLs
urls = {
    "holidays": "https://www.dropbox.com/s/kaovdndtevcvt83/holidays.csv?dl=1",
    "items": "https://www.dropbox.com/s/wh6amz4um7ieyqz/items.csv?dl=1",
    "oil": "https://www.dropbox.com/s/ze6of1xqwslt8jb/oil.csv?dl=1",
    "sales_small": "https://www.dropbox.com/s/uiojfrc5c20gyrl/sales_small.csv?dl=1",
    "stores": "https://www.dropbox.com/s/cwy6z0b7rhsnrxb/stores.csv?dl=1",
    "train_small": "https://www.dropbox.com/s/wwaoga17z70jb6l/train_small.csv?dl=1",
    "transactions": "https://www.dropbox.com/s/2bxto9wnetwnvqd/transactions.csv?dl=1",
}

# Download files
for name, url in urls.items():
    urllib.request.urlretrieve(url, f"{name}.csv")

# Connect to database
con = duckdb.connect(database=':memory:')

# Create tables
con.execute("CREATE OR REPLACE TABLE holidays AS SELECT * FROM 'holidays.csv';")
con.execute("CREATE OR REPLACE TABLE oil AS SELECT * FROM 'oil.csv';")
con.execute("CREATE OR REPLACE TABLE transactions AS SELECT * FROM 'transactions.csv';")
con.execute("CREATE OR REPLACE TABLE stores AS SELECT * FROM 'stores.csv';")
con.execute("CREATE OR REPLACE TABLE items AS SELECT * FROM 'items.csv';")
con.execute("CREATE OR REPLACE TABLE sales AS SELECT * FROM 'sales_small.csv';")
con.execute("CREATE OR REPLACE TABLE train AS SELECT * FROM 'train_small.csv';")

# Data exploration and Data Transformation

You can use database SQL queries to  perform data transformations over tables.


In [5]:
con.execute("describe;").df()

,table_name,column_names,column_types,temporary
0,holidays,"[date, htype, locale, locale_name, transferred...","[BIGINT, BIGINT, BIGINT, BIGINT, BIGINT, BIGINT]",False
1,items,"[item_nbr, family, class, perishable, f1]","[BIGINT, BIGINT, BIGINT, BIGINT, BIGINT]",False
2,oil,"[date, dcoilwtico, f3]","[BIGINT, BIGINT, BIGINT]",False
3,sales,"[item_nbr, unit_sales, onpromotion, tid, Y]","[BIGINT, DOUBLE, BIGINT, BIGINT, DOUBLE]",False
4,stores,"[store_nbr, city, state, stype, cluster, f4]","[BIGINT, BIGINT, BIGINT, BIGINT, BIGINT, BIGINT]",False
5,train,"[Y, onpromotion, htype, locale, locale_name, t...","[DOUBLE, BIGINT, BIGINT, BIGINT, BIGINT, BIGIN...",False
6,transactions,"[date, store_nbr, transactions, tid, f5]","[BIGINT, BIGINT, BIGINT, BIGINT, BIGINT]",False


# Building a Join Graph

First, you need to specify the join graph, which includes the relations and features you want to use, as well as how these relations should be joined. For example,

```
dataset.add_join("sales", "items", ["item_nbr"], ["item_nbr"])
```
indicates that the join condition between the "sales" and "items" tables is based on the item_nbr attribute, where sales.item_nbr = items.item_nbr. By specifying the join conditions in this manner, you can define how the tables in your dataset are related and should be combined.

In [6]:
exe = DuckdbExecutor(con, debug=False)
dataset = JoinGraph(exe=exe)
dataset.add_relation("sales", [], y = 'Y')
dataset.add_relation("holidays", ["htype", "locale", "locale_name", "transferred","f2"])
dataset.add_relation("oil", ["dcoilwtico","f3"])
dataset.add_relation("transactions", ["transactions","f5"])
dataset.add_relation("stores", ["city","state","stype","cluster","f4"])
dataset.add_relation("items", ["family","class","perishable","f1"])
dataset.add_join("sales", "items", ["item_nbr"], ["item_nbr"])
dataset.add_join("sales", "transactions", ["tid"], ["tid"])
dataset.add_join("transactions", "stores", ["store_nbr"], ["store_nbr"])
dataset.add_join("transactions", "holidays", ["date"], ["date"])
dataset.add_join("holidays", "oil", ["date"], ["date"])

dataset

# Training a Decision Tree with JoinBoost

Once you have defined the join graph, you can train a decision tree over it using JoinBoost.

In [7]:
depth = 3
reg = DecisionTree(learning_rate=1, max_leaves=2 ** depth, max_depth=depth)

reg.fit(dataset)
reg.compute_rmse('train')[0]

joinboost_tmp_76


2535.0608391585865

# Training Gradient Boosting with JoinBoost

After training a decision tree, you can train a Gradient Boosting model with JoinBoost for further improvement. In this example, we train a Gradient Boosting model with 10 iterations.

As demonstrated, Gradient Boosting can further reduce the RMSE.

In [8]:
from joinboost.app import GradientBoosting

# Set the parameters for the Gradient Boosting model
reg = GradientBoosting(learning_rate=1, max_leaves=2 ** depth, max_depth=depth, iteration=10)

# Train the Gradient Boosting model on the dataset
reg.fit(dataset)

# Compute the Root Mean Square Error (RMSE) on the training set
train_rmse = reg.compute_rmse('train')[0]
print("Training RMSE for Gradient Boosting:", train_rmse)

joinboost_tmp_828
Training RMSE for Gradient Boosting: 786.8773440907672


# Exploring JoinBoost Internals

Internally, JoinBoost translates machine learning logic into SQL and executes SQL queries directly in your database without the need for data movement.

To see the SQL queries generated by JoinBoost, you can enable debug mode when creating an executor instance:

In [9]:
# Create SparkExecutor instance with debug mode enabled
exe = DuckdbExecutor(con, debug=True)

dataset = JoinGraph(exe=exe)
dataset.add_relation("sales", [], y = 'Y')
dataset.add_relation("holidays", ["htype", "locale", "locale_name", "transferred","f2"])
dataset.add_relation("oil", ["dcoilwtico","f3"])
dataset.add_relation("transactions", ["transactions","f5"])
dataset.add_relation("stores", ["city","state","stype","cluster","f4"])
dataset.add_relation("items", ["family","class","perishable","f1"])
dataset.add_join("sales", "items", ["item_nbr"], ["item_nbr"])
dataset.add_join("sales", "transactions", ["tid"], ["tid"])
dataset.add_join("transactions", "stores", ["store_nbr"], ["store_nbr"])
dataset.add_join("transactions", "holidays", ["date"], ["date"])
dataset.add_join("holidays", "oil", ["date"], ["date"])

# Set the parameters for the decision tree
depth = 3
reg = DecisionTree(learning_rate=1, max_leaves=2 ** depth, max_depth=depth)

# Train the decision tree on the dataset
reg.fit(dataset)

# Compute the Root Mean Square Error (RMSE) on the training set
train_rmse = reg.compute_rmse('train')[0]
print("Training RMSE:", train_rmse)

PRAGMA table_info(sales)
0.005413532257080078
[(0, 'item_nbr', 'BIGINT', False, None, False), (1, 'unit_sales', 'DOUBLE', False, None, False), (2, 'onpromotion', 'BIGINT', False, None, False), (3, 'tid', 'BIGINT', False, None, False), (4, 'Y', 'DOUBLE', False, None, False)]
PRAGMA table_info(holidays)
0.0008273124694824219
[(0, 'date', 'BIGINT', False, None, False), (1, 'htype', 'BIGINT', False, None, False), (2, 'locale', 'BIGINT', False, None, False), (3, 'locale_name', 'BIGINT', False, None, False), (4, 'transferred', 'BIGINT', False, None, False), (5, 'f2', 'BIGINT', False, None, False)]
PRAGMA table_info(oil)
0.0007359981536865234
[(0, 'date', 'BIGINT', False, None, False), (1, 'dcoilwtico', 'BIGINT', False, None, False), (2, 'f3', 'BIGINT', False, None, False)]
PRAGMA table_info(transactions)
0.0009200572967529297
[(0, 'date', 'BIGINT', False, None, False), (1, 'store_nbr', 'BIGINT', False, None, False), (2, 'transactions', 'BIGINT', False, None, False), (3, 'tid', 'BIGINT', Fals